#RNA family characterization

Application scenario: we want to characterize the sequence patterns and structure of a set of RNA seqeunces

Aquire data.

In [1]:
rfam_id = 'RF00005'
#rfam_id = 'RF01051'
#rfam_id = 'RF00008'
#rfam_id = 'RF00175'
#rfam_id = 'RF00059'
#rfam_id = 'RF00008'
def rfam_url(family_id):
    return 'http://rfam.xfam.org/family/%s/alignment?acc=%s&format=fastau&download=0'%(family_id,family_id)

In [2]:
import ConstraintFinder as cf

ImportError: No module named ConstraintFinder

Convert the fasta database into eden graphs.

In [ ]:
def create_sequence_pool(rfam_id):
    from eden.converter.fasta import fasta_to_sequence
    iterable = fasta_to_sequence(rfam_url(rfam_id))
    sequence_pool = []
    for fasta in iterable:
        sequence_pool.append(fasta[1])
    return sequence_pool
#print create_sequence_pool(rfam_id)

In [ ]:
from eden.converter.fasta import fasta_to_sequence
from eden.converter.rna.rnafold import rnafold_to_eden

iterable_pos = fasta_to_sequence( rfam_url(rfam_id) )
members = 0
for item in iterable_pos:
    members += 1
print members

In [ ]:
iterable_pos = fasta_to_sequence( rfam_url(rfam_id) )

from itertools import islice, tee
iterable_pos, iterable_pos_ = tee(iterable_pos)

from eden.modifier.seq import seq_to_seq,  shuffle_modifier
iterable_neg = seq_to_seq( iterable_pos_ , modifier=shuffle_modifier, times=3, order=2 )

#Positive sample graphs.
graphs_pos = rnafold_to_eden( iterable_pos )
#Negative sample graphs.
graphs_neg = rnafold_to_eden( iterable_neg )


Split test and train graph sets into fixed test and train portions. (20% for training)

In [ ]:
from itertools import islice, chain
from eden.util import fit,estimate
from eden.graph import Vectorizer
import RNADesignTools as rdt

antaRNA_param_file = '/home/kohvaeip/RLS/lib/antaRNA/antaRNA.ini'

In [ ]:
vectorizer = Vectorizer( complexity = 2 )

In [3]:
%%time
estimator = fit(graphs_pos , graphs_neg , vectorizer, n_jobs=1, cv=3)

NameError: name 'fit' is not defined

In [ ]:
%%time
iterable_pos_ = fasta_to_sequence( rfam_url(rfam_id) )
graphs = rnafold_to_eden( iterable_pos_ )
agraphs = vectorizer.annotate(graphs, estimator)
# Design the batch of new sequences.
#dsi = rdt.DesignRNA(antaRNA_param_file , graphs , vectorizer, estimator, 0, 3, 0, 3)

In [ ]:
import ConstraintFinder as cf
reload(cf)

#Generate graph list ******
for g in agraphs:
    print cf.antaRNA_dot_struct(g,-0.85, 3, 3)
    print cf.antaRNA_constraint_string(g, -0.2 , 1 )
#antaRNA_dot_struct(g,threshold, importance_adjacency = 1, unpaired_adjacency = 1)
#antaRNA_constraint_string(g, threshold, adjacency = 1 , padding= 'N')
#generate_antaRNA_constraints(graphs, Cseq_threshold, Cseq_adjacency, dotnot_threshold, dotnot_adjacency, unpaired_adjacency)
#compute_gc_content

# GNNGNUNUNGNUNANUNNUANANCNNNNGNNUNNNAANCNNNNNNNNNNGGUUCNAAUCCNNNNAUCNNCA
# (((((((..(((A.......A))).(((((.......)))))....(((((.......))))))))))))A
# -((((((----(---------)----((-(-------)-))------((-----------))-))))))--